# Transform

## Import

In [60]:
import pandas as pd
import numpy as np
import os
import json

In [61]:
# Pfad
TEMPLATE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "_templates"))

In [62]:
# Lade die Pickle-Dateien
partei_df = pd.read_pickle(os.path.join(TEMPLATE_PATH, "parteien.pickle"))
wahlen = pd.read_pickle(os.path.join(TEMPLATE_PATH, "politbarometer.pickle"))

## Analyse

In [63]:
print(partei_df.shape, wahlen.shape)

(37948064, 4) (568, 8)


In [64]:
partei_df=partei_df.dropna()

In [65]:
partei_df.head()

,count,word,date,paper
1232,1,cdu,2021-04-01,sz
1771,1,cdu,2021-04-01,sz
2940,2,cdu,2021-04-01,zeit
3451,1,cdu,2021-04-01,zeit
6018,1,grünen,2021-04-01,faz


In [66]:
# Eine Kopie der Daten erstellen
wahlen = wahlen.copy()
wordcount = partei_df.copy()

## Wordcount df's erstellen

In diesem Code werden die gesuchten Parteinamen ab einem bestimmten Datum gefiltert, aggregiert und in verschiedene Zeitformate umgewandelt, um eine übersichtliche Auswertung zu ermöglichen.

In [67]:
# Einschränken auf Daten ab 2021-04-01 und Index zurücksetzen
wordcount = wordcount[wordcount['date'] >= '2021-04-01'].reset_index(drop=True)
wordcount = wordcount[wordcount["date"] <= "2024-12-01"].reset_index(drop=True)

# Gruppieren nach Datum und Wort sowie Aufsummieren der 'count'-Werte
wordcount = wordcount.groupby(['date', 'word']).agg({'count': 'sum'}).reset_index()

# Konvertieren der 'date'-Spalte in ein Datumsformat
wordcount["date"] = pd.to_datetime(wordcount["date"])

# Hinzufügen einer Spalte für das Jahr und den Monat (Format 'YYYY-MM')
wordcount["month"] = wordcount["date"].dt.strftime('%Y-%m')

# Hinzufügen einer Spalte für das Quartal
wordcount["quarter"] = wordcount["date"].dt.to_period('Q')

# Ermitteln der Gesamtanzahl pro Wort
word_counts = wordcount.groupby("word")["count"].sum()

In [68]:
word_counts

word
afd       37128
bsw        4873
cdu       33175
fdp       22829
grünen    27395
linke     22414
spd       33849
Name: count, dtype: int64

Hier werden die Parteien definiert, die in der Visualisierung zusammen dargestellt werden sollen

In [69]:
# Liste der Wörter, die behalten werden sollen
words_to_keep = ["grünen", "afd"]
wordkeep = ["spd", "cdu"]
# Filtern des DataFrames, um nur die gewünschten Wörter zu behalten
kleine_parties = wordcount[wordcount["word"].isin(words_to_keep)]
große_parties = wordcount[wordcount["word"].isin(wordkeep)]

## Wahlen (Politbarometer) df erstellen

In diesem Code werden die Barometerdaten auf einen bestimmten Zeitraum eingeschränkt, ihre Spalten vereinheitlicht und zusätzliche Zeitintervalle für die Analyse hinzugefügt.

In [70]:
# Konvertieren der 'date'-Spalte in ein Datumsformat
wahlen["date"] = pd.to_datetime(wahlen["date"])

# Daten auf Zeitraum vom 01.04.2021 bis 01.12.2024 beschränken
wahlen = wahlen[wahlen["date"] >= "2021-04-01"].reset_index(drop=True)
wahlen = wahlen[wahlen["date"] <= "2024-12-01"].reset_index(drop=True)

# Spaltennamen vereinheitlichen und neue Zeitspalten hinzufügen
wahlen.columns = wahlen.columns.str.lower()
wahlen["quarter"] = wahlen["date"].dt.to_period("Q").astype(str)
wahlen["month"] = wahlen["date"].dt.to_period("M").astype(str)

# Konvertieren der CDU-Spalte in Integer
wahlen["cdu"] = wahlen["cdu"].astype(int)

# Wandle alle spezifischen Spalten von float zu integer um
wahlen[['cdu', 'spd', 'grüne', 'fdp', 'linke', 'afd', 'bsw']] = wahlen[['cdu', 'spd', 'grüne', 'fdp', 'linke', 'afd', 'bsw']].fillna(0).astype(int)

# Länge jeder Spalte ausgeben
for key, value in wahlen.items():
    print(f'Länge von {key}:', len(value))


Länge von date: 72
Länge von cdu: 72
Länge von spd: 72
Länge von grüne: 72
Länge von fdp: 72
Länge von linke: 72
Länge von afd: 72
Länge von bsw: 72
Länge von quarter: 72
Länge von month: 72


## json-Datei erstellen fuer d3 visualisierung


In [71]:
df = partei_df.copy()

# Datum umwandeln und sortieren
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(by=["word", "date"])

# Kumulative Summe berechnen
df["cumulative_count"] = df.groupby("word")["count"].cumsum()

# Datum in JSON-kompatibles Format umwandeln
df["date"] = df["date"].dt.strftime("%Y-%m-%d")

# Gruppieren und JSON-strukturierte Daten erstellen
grouped_data = df.groupby("word").apply(
    lambda group: group[["date", "cumulative_count"]].to_dict(orient="records")
).to_dict()

## Export

In [72]:
# Df's als pickle speichern
kleine_parties.to_pickle(os.path.join(TEMPLATE_PATH, "kleine.pickle"))
große_parties.to_pickle(os.path.join(TEMPLATE_PATH, "große.pickle"))
wahlen.to_pickle(os.path.join(TEMPLATE_PATH, "wahlen.pickle"))
partei_df.to_pickle(os.path.join(TEMPLATE_PATH, "alle_parteien.pickle"))
word_counts.to_pickle(os.path.join(TEMPLATE_PATH, "kumuliert.pickle"))

In [73]:
# JSON-Pfad fuer d3 Plot 
json_path = os.path.join(TEMPLATE_PATH, "cumulative_parteien.json")

# JSON-Datei speichern
with open(json_path, "w") as f:
    json.dump(grouped_data, f)

print(f"Kumulierte JSON-Daten gespeichert unter: {json_path}")

Kumulierte JSON-Daten gespeichert unter: c:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\_templates\cumulative_parteien.json
